In [1]:
from KAN_Impl import KAN
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import torch
import mplhep as hep
hep.style.use("CMS")
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
# device = 'cpu'

In [2]:
grid_range=[-8, 8]

In [ ]:
X_train=torch.from_numpy(np.load('data/X_train_val.npy')).float().to(device)
y_train=torch.from_numpy(np.load('data/y_train_val.npy')).float().to(device).argmax(dim=1)
X_test=torch.from_numpy(np.load('data/X_test.npy')).float().to(device)
y_test=torch.from_numpy(np.load('data/y_test.npy')).float().to(device).argmax(dim=1)

# Create TensorDataset objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader objects
batch_size = 64  # Adjust this based on your available memory
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = KAN([16,4,5], grid_size=30, spline_order=3, grid_eps=0.05, base_activation=nn.GELU, grid_range=grid_range, quantize=True, tp=16, fp=6, lut_res=256)
print(sum(p.numel() for p in model.parameters()))

model.to(device)
# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
# Define learning rate scheduler
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

training_loss = []
testing_loss = []

# Define loss
criterion = nn.CrossEntropyLoss()
for epoch in range(1):
    # Train
    model.train()
    epoch_train_loss = 0  # Initialize loss for the epoch
    total_batches = 0
    with tqdm(trainloader) as pbar:
        for i, (inputs, labels) in enumerate(pbar):
            inputs = inputs.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels.to(device))
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()
            total_batches += 1

            accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])
    
    average_train_loss = epoch_train_loss / total_batches
    training_loss.append(average_train_loss)  # Record the average training loss

    # Validation
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            output = model(inputs)
            val_loss += criterion(output, labels.to(device)).item()
            val_accuracy += (
                (output.argmax(dim=1) == labels.to(device)).float().mean().item()
            )
    val_loss /= len(testloader)
    val_accuracy /= len(testloader)
    testing_loss.append(val_loss)

    # Update learning rate
    scheduler.step()

    print(
        f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
    )

In [ ]:
print("Grid Range: ", grid_range)

#Loop through each layer of the activation function
for i in range(len(model.layers)):
    layer = model.layers[i]
    print("Layer: ", i)

    #Create dummy input
    array = np.linspace(grid_range[0], grid_range[1], 256)
    stacked_array = np.hstack([[array]*layer.in_features]).T
    x=torch.from_numpy(stacked_array).float().to(device)

    #Loop through each activation function
    for j in range(layer.in_features):
        for k in range(layer.out_features):
            base_output = layer.base_activation(x)[:,j]*layer.base_weight[k,j]
            spline_output = F.linear(layer.b_splines(x)[:,j,:], layer.scaled_spline_weight[k,j,:])

            output = base_output + spline_output

            print(output.cpu().detach().numpy().flatten().shape)
            print(x.cpu().detach().numpy().shape)

            plt.plot(x[:,j].cpu().detach().numpy(), output.cpu().detach().numpy(), label=f"Layer {i}. Node {j} -> {k}")
            plt.xlabel("Input Value")
            plt.ylabel("Output Value")
            plt.legend()
            plt.show()

In [ ]:
print("Grid Range: ", grid_range)

test_LUT = []

#Loop through each layer of the activation function
for i in range(len(model.layers)):
    layer = model.layers[i]
    print("Layer: ", i)

    #Create dummy input
    array = np.linspace(grid_range[0], grid_range[1], 256)
    stacked_array = np.hstack([[array]*layer.in_features]).T
    x=torch.from_numpy(stacked_array).float().to(device)

    #Loop through each activation function
    for j in range(layer.in_features):
        for k in range(layer.out_features):
            base_output = layer.base_activation(x)[:,j]*layer.base_weight[k,j]
            spline_output = F.linear(layer.b_splines(x)[:,j,:], layer.scaled_spline_weight[k,j,:])

            output = base_output + spline_output

            test_LUT.append(output.cpu().detach().numpy())

            print(output.cpu().detach().numpy().flatten().shape)
            print(x.cpu().detach().numpy().shape)

            plt.plot(x[:,j].cpu().detach().numpy(), output.cpu().detach().numpy(), label=f"Layer {i}. Node {j} -> {k}")
            plt.xlabel("Input Value")
            plt.ylabel("Output Value")
            plt.legend()
            plt.show()

In [6]:
a = test_LUT[0]

In [ ]:
a.shape

In [ ]:
plt.plot(a)

In [ ]:
a